In [1]:
import sys
import os
import nbimporter
import copy
import math
import numpy

import json
from collections import OrderedDict
import pprint

# 外皮情報LV4から簡易熱負荷計算入力情報へのコンバート

## 1. Convert Gdata

#### Outline

'Gdata'の書き出し

#### Function

In [2]:
def convert_gdata(d):
    
    d['Gdata'] = {'Region': d['Common']['Region']}
    
    return d['Gdata']

## 2. Convert Rooms

### 床下空間の床面積の取得

#### Outline

方位が'Bottom'である床下空間の外皮の部位と、床下空間の土間床を、床下空間の床とみなして、床下空間の床面積を取得する。  
床下空間に属する部位があるが、床下空間の床面積が0㎡の場合、エラーとする。

#### Function

In [3]:
def get_area_underfloor(d):
    
    area_underfloor = 0
    make_underfloor = False
    
    for x in ['Walls', 'Windows', 'Doors'] :
        if (x in d) == True :
            for y in d[x] :
                if y['space'] == 'underfloor' and y['directioin'] == 'Bottom' :
                    area_underfloor = area_underfloor + y['area']
    
    if ('Earthfloors' in d) == True :
        for x in d['Earthfloors'] :
            if x['space'] == 'underfloor' :
                area_underfloor = area_underfloor + x['area']

    for x in ['Walls', 'Windows', 'Doors', 'Heatbridges', 'EarthfloorPerimeters', 'Earthfloors', 'Innerwalls'] :
        if (x in d) == True :
            for y in d[x] :
                if y['space'] == 'underfloor' :
                    make_underfloor = True
            else :
                continue
            break

    if area_underfloor == 0 and make_underfloor == True :
        raise ValueError
    
    return area_underfloor

### Roomsの設定

#### Outline

Roomsの初期設定  

#### Function

In [4]:
def get_d_main_init(area_main, h_room, Nvent, area_total, area_other):

    d_main_init = {
        'name': 'main',
        'Vol' : area_main*h_room,
        'Vent' : {
            'winter': Nvent*area_total*h_room*area_main/ (area_main + area_other),
            'inter':  Nvent*area_total*h_room*area_main/ (area_main + area_other),
            'summer': Nvent*area_total*h_room*area_main/ (area_main + area_other)}
        ,
        'Inf' : {'winter': 0,'inter': 0,'summer': 0},
        'NextVent': [],
        'LinearHeatBridge': [],
        'EarthfloorPerimeter': [],
        'Surface': []
    }    
    
    return d_main_init

In [5]:
def get_d_other_init(area_other, h_room, Nvent, area_total, area_main):

    d_other_init = {
        'name': 'other',
        'Vol' : area_other*h_room,
        'Vent' : {
            'winter': Nvent*area_total*h_room*area_other/ (area_main + area_other),
            'inter':  Nvent*area_total*h_room*area_other/ (area_main + area_other),
            'summer': Nvent*area_total*h_room*area_other/ (area_main + area_other)}
        ,
        'Inf' : {'winter': 0,'inter': 0,'summer': 0},
        'NextVent': [],
        'LinearHeatBridge': [],
        'EarthfloorPerimeter': [],
        'Surface': []
    }    
    
    return d_other_init

In [6]:
def get_d_nonliving_init(area_nonliving, h_room, Nvent, area_total, area_main, area_other):

    next_vent = []
    if area_main > 0 :
        next_vent.append({'Windward_roomname': 'main',
                          'winter': Nvent*area_total*h_room*area_main/ (area_main + area_other),
                          'inter':  Nvent*area_total*h_room*area_main/ (area_main + area_other),
                          'summer': Nvent*area_total*h_room*area_main/ (area_main + area_other)})
    if area_other > 0 :
        next_vent.append({'Windward_roomname': 'other',
                          'winter': Nvent*area_total*h_room*area_other/ (area_main + area_other),
                          'inter':  Nvent*area_total*h_room*area_other/ (area_main + area_other),
                          'summer': Nvent*area_total*h_room*area_other/ (area_main + area_other)})

    d_nonliving_init = {
        'name': 'nonliving',
        'Vol' : area_nonliving*h_room,
        'Vent' : { 'winter': 0, 'inter': 0, 'summer': 0},
        'Inf' : {'winter': 0,'inter': 0,'summer': 0},
        'NextVent': next_vent,
        'LinearHeatBridge': [],
        'EarthfloorPerimeter': [],
        'Surface': []
    }   
    
    return d_nonliving_init

In [7]:
def get_d_underfloor_init(area, h, Nvent):

    d_underfloor_init = {
        'name': 'underfloor',
        'Vol' : area*h,
        'Vent' : {
            'winter': Nvent*area*h,
            'inter':  Nvent*area*h,
            'summer': Nvent*area*h}
        ,
        'Inf' : {'winter': 0,'inter': 0,'summer': 0},
        'NextVent': [],
        'LinearHeatBridge': [],
        'EarthfloorPerimeter': [],
        'Surface': []
    }

    return d_underfloor_init

In [8]:
def get_rooms_initial(d):
    
    # 床面積の取得
    area_total = d['Common']['TotalFloorArea'] 
    area_main = d['Common']['MainOccupantRoomFloorArea']
    area_other = d['Common']['OtherOccupantRoomFloorArea'] 
    area_nonliving = area_total - area_main - area_other
    area_underfloor = get_area_underfloor(d)
    
    # 天井高の設定
    h_room = 2.4
    h_underfloor = 0.4
    
    # 換気回数の設定
    Nvent = 0.5
    Nvent_underfloor = 5.0

    d['Rooms'] = []

    # 主たる居室の作成
    if area_main > 0 :
        d['Rooms'].append(get_d_main_init(area_main, h_room, Nvent, area_total, area_other))
        
    # その他の居室の作成    
    if area_other > 0 :
        d['Rooms'].append(get_d_other_init(area_other, h_room, Nvent, area_total, area_main))
        
    # 非居室の作成
    if area_nonliving > 0 :
        d['Rooms'].append(get_d_nonliving_init(area_nonliving, h_room, Nvent, area_total, area_main, area_other))
        
    # 床下空間の作成
    if area_underfloor > 0 :
        d['Rooms'].append(get_d_underfloor_init(area_underfloor, h_underfloor, Nvent_underfloor))
    
    return d['Rooms']

### 表面熱伝達抵抗の取得

#### Outline

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  
　　　　　付録 A 住宅の平均熱貫流率算出に用いる建材等の熱物性値等   
　　　　　　表 3.1 表面熱伝達抵抗   
　　　　　　表 3.2 表面熱伝達抵抗（界壁・界床の場合） 

<div style="text-align: center;"> 表3.1 表面熱伝達抵抗 </div>

| 部位 | 熱的境界内側（室内側）の<br>表面熱伝達抵抗[㎡K/W] | 熱的境界外側（外気側）の表面熱伝達抵抗[㎡K/W]<br>外気に直接接する場合 | 熱的境界外側（外気側）の表面熱伝達抵抗[㎡K/W]<br>左記以外の場合 |
|---|---|---|---|
| 屋根 | 0.09 | 0.04 | 0.09（通気層等） |
| 天井 | 0.09 | - | 0.09（小屋裏等） |
| 外壁 | 0.11 | 0.04 | 0.11（通気層等） |
| 床 | 0.15 | 0.04 | 0.15(床裏等) |

<div style="text-align: center;"> 表3.2 表面熱伝達抵抗（界壁・界床の場合） </div>

| 部位 | 対象住戸の室内側表面熱伝達抵抗[㎡K/W] | 隣接住戸の室内側表面熱伝達抵抗[㎡K/W] |
|---|---|---|
| 界壁 | 0.11 | 0.11 |
| 上階側界床 | 0.09 | 0.09 |
| 下階側界床 | 0.15 | 0.15 |

#### Function

In [9]:
def get_OutHeatTrans(type, IsInContactWithOutsideAir):
    
    if IsInContactWithOutsideAir == True :
        Ro = {
            'Roof'            : 0.040,
            'Wall'            : 0.040,
            'Floor'           : 0.040
        }[type]
    else :
        Ro = {
            'Roof'            : 0.090,
            'Ceiling'         : 0.090,
            'Wall'            : 0.110,
            'Floor'           : 0.150,
            'BoundaryCeiling' : 0.090,
            'BoundaryWall'    : 0.110,
            'BoundaryFloor'   : 0.150,
            'InnerCeiling'    : 0.150,
            'InnerWall'       : 0.110,
            'InnerFloor'      : 0.090,
            'GroundFloor'     : 0.150
        }[type]
    
    return 1/Ro

In [10]:
def get_InHeatTrans_from_type(type):
    
    Ri = {
        'Roof'            : 0.090,
        'Ceiling'         : 0.090, 
        'Wall'            : 0.110, 
        'Floor'           : 0.150, 
        'Earthfloor'      : 0.150, 
        'BoundaryCeiling' : 0.090,
        'BoundaryWall'    : 0.110,
        'BoundaryFloor'   : 0.150,
        'InnerCeiling'    : 0.090,
        'InnerWall'       : 0.110,
        'InnerFloor'      : 0.150,
        'GroundFloor'     : 0.150
    }[type]
    
    return 1/Ri

In [11]:
def get_InHeatTrans_from_direction(direction):
    
    Ri = {
        'Top':     0.090,
        'N':       0.110,
        'NE':      0.110,
        'E':       0.110,
        'SE':      0.110,
        'S':       0.110,
        'SW':      0.110,
        'W':       0.110,
        'NW':      0.110,
        'Bottom':  0.150
    }[direction]
    
    return 1/Ri

### Boundaryの設定

#### Outline

方位情報より方位角・傾斜角を取得する。

#### Function

In [12]:
def get_directionangle_from_direction(direction):
    
    Direction_to_DirectionAngle = {
        'Top': None,
        'N':   180,
        'NE': -135,
        'E':   -90,
        'SE':  -45,
        'S':     0,
        'SW':   45,
        'W':    90,
        'NW':  135,
        'Bottom':          None,
        'ClosedSpace':     None,
        'OpenBackFloor':   None,
        'ResidenceSpace':  None,
        'ClosedBackFloor': None
    }
    
    return Direction_to_DirectionAngle[direction]

In [13]:
def get_inclinationangle_from_direction(direction):
    
    Direction_to_InclinationAngle = {
        'Top': 0,
        'N':  90,
        'NE': 90,
        'E':  90,
        'SE': 90,
        'S':  90,
        'SW': 90,
        'W':  90,
        'NW': 90,
        'Bottom': 180,
        'ClosedSpace':     None,
        'OpenBackFloor':   None,
        'ResidenceSpace':  None,
        'ClosedBackFloor': None
    }
    
    return Direction_to_InclinationAngle[direction]

#### Outline

方位情報より隣室温度差係数を取得する。

#### Function

In [14]:
def get_tempdifferentfactor_from_direction(region, direction):
    
    Direction_to_NextSpace = {
        'Top':     'outside',
        'N':       'outside',
        'NE':      'outside',
        'E':       'outside',
        'SE':      'outside',
        'S':       'outside',
        'SW':      'outside',
        'W':       'outside',
        'NW':      'outside',
        'Bottom':  'outside',
        'ClosedSpace':     'underfloor',
        'OpenBackFloor':   'underfloor',
        'ResidenceSpace':  'next_room',
        'ClosedBackFloor': 'next_room'
    }    
    
    NextSpace_to_TempDifferentFactor = {
        # 外気又は外気に通じる空間(小屋裏・天井裏・共用部・屋内駐車場・メーターボックス・エレベーターシャフト等)
        'outside'    : {1: 1.00, 2: 1.00, 3: 1.00, 4: 1.00, 5: 1.00, 6: 1.00, 7: 1.00, 8: 1.00 }, 
        # 外気に通じていない空間(昇降機室、共用機械室、倉庫等)又は外気に通じる床裏
        'underfloor' : {1: 0.70, 2: 0.70, 3: 0.70, 4: 0.70, 5: 0.70, 6: 0.70, 7: 0.70, 8: 0.70 }, 
        # 住戸、住戸と同様の熱的環境の空問(空調された共用部等)又は外気に通じていない床裏(ピット等）
        'next_room'  : {1: 0.05, 2: 0.05, 3: 0.05, 4: 0.15, 5: 0.15, 6: 0.15, 7: 0.15, 8: 0.15 }  
    }
    
    return  NextSpace_to_TempDifferentFactor[Direction_to_NextSpace[direction]][region]

#### Outline

Boundaryを設定する。

#### Function

In [1]:
def get_boundary_for_outer_skin(region, nextspace, direction):
    
    Type = {
        'Top':     'Outdoor',
        'N':       'Outdoor',
        'NE':      'Outdoor',
        'E':       'Outdoor',
        'SE':      'Outdoor',
        'S':       'Outdoor',
        'SW':      'Outdoor',
        'W':       'Outdoor',
        'NW':      'Outdoor',
        'Bottom':  'Outdoor',
        'ClosedSpace':     'DeltaTCoeff',
        'OpenBackFloor':   'DeltaTCoeff',
        'ResidenceSpace':  'DeltaTCoeff',
        'ClosedBackFloor': 'DeltaTCoeff'
    }    
    
    boundary = {
        'Outdoor': {
            'Name': direction,
            'Type': 'Outdoor',
            'DirectionAngle': get_directionangle_from_direction(direction if nextspace=='Outdoor' else nextspace),
            'InclinationAngle': get_inclinationangle_from_direction(direction if nextspace=='Outdoor' else nextspace),
            'GroundReflectRate': 0.1
        },
        'DeltaTCoeff': {
            'Name': direction,
            'Type': 'DeltaTCoeff',
            'TempDifferFactor': get_tempdifferentfactor_from_direction(region, direction if nextspace =='Outdoor' else nextspace)
        }
    }[Type[direction if nextspace=='Outdoor' else nextspace]]

    return boundary

In [16]:
def get_boundary_for_inner_skin(type, nextspace):
    
    boundary = {
        'Name': 'NextRoom' + '_' + nextspace,
        'Type': 'NextRoom',
        'RoomName': nextspace
    }
    
    return boundary

In [1]:
def get_boundary_for_earthfloorperimeter(region, direction):
    
    Type = {
        'Top':     'Outdoor',
        'N':       'Outdoor',
        'NE':      'Outdoor',
        'E':       'Outdoor',
        'SE':      'Outdoor',
        'S':       'Outdoor',
        'SW':      'Outdoor',
        'W':       'Outdoor',
        'NW':      'Outdoor',
        'Bottom':  'Outdoor',
        'ClosedSpace':     'DeltaTCoeff',
        'OpenBackFloor':   'DeltaTCoeff',
        'ResidenceSpace':  'DeltaTCoeff',
        'ClosedBackFloor': 'DeltaTCoeff'
    }    
    
    boundary = {
        'Outdoor': {
            'Name': direction,
            'Type': 'Outdoor'
        },
        'DeltaTCoeff': {
            'Name': direction,
            'Type': 'DeltaTCoeff',
            'TempDifferFactor': get_tempdifferentfactor_from_direction(region, direction)
        }
    }[Type[direction]]

    return boundary

### directionの設定

#### Outline

方位（Top/N/NE/E/SE/S/SW/W/NW/Bottom）より方位（Upward/Vertical/Downward）を決定する。  
方位（Top/N/NE/E/SE/S/SW/W/NW/Bottom）にClosedSpace/OpenBackFloor/ResidenceSpace/ClosedBackFloorが含まれるが、方位と温度差係数を別の変数として扱うべき。

#### Function

In [18]:
def get_direction_from_direction_for_outer_skin(direction):
    
    return {
        'Top':     'Downward',
        'N':       'Vertical',
        'NE':      'Vertical',
        'E':       'Vertical',
        'SE':      'Vertical',
        'S':       'Vertical',
        'SW':      'Vertical',
        'W':       'Vertical',
        'NW':      'Vertical',
        'Bottom':  'Upward',
        'ClosedSpace':     'Vertical', # 暫定的な処理。方位と温度差係数を別の変数として扱うべき。
        'OpenBackFloor':   'Upward',    # 暫定値な処理。方位と温度差係数を別の変数として扱うべき。
        'ResidenceSpace':  'Vertical', # 暫定値な処理。方位と温度差係数を別の変数として扱うべき。
        'ClosedBackFloor': 'Upward'     # 暫定値な処理。方位と温度差係数を別の変数として扱うべき。             
    }[direction]

#### Outline

内壁種類（InnerCeiling/InnerWall/InnerFloo/GroundFloor）と室より方位（Upward/Vertical/Downward）を決定する。

#### Function

In [19]:
def get_direction_from_direction_for_inner_skin(space, type):
    
    return {
        'main':       {'InnerCeiling': 'Downward', 'InnerWall': 'Vertical', 'InnerFloor': 'Upward', 'GroundFloor': 'Upward'},
        'other':      {'InnerCeiling': 'Downward', 'InnerWall': 'Vertical', 'InnerFloor': 'Upward', 'GroundFloor': 'Upward'},
        'nonliving':  {'InnerCeiling': 'Downward', 'InnerWall': 'Vertical', 'InnerFloor': 'Upward', 'GroundFloor': 'Upward'},
        'underfloor': {'InnerCeiling': None,       'InnerWall': None,        'InnerFloor': None,     'GroundFloor': 'Downward'}
    }[space][type]

### UA値入力の場合の壁体の設定

#### Outline

UA値より壁体構成を決定する。

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [20]:
class Material:
    
    def __init__( self, name, thick, cond, spech ):
        self.name   = name
        self.thick  = thick   # thickness, m
        self.cond   = cond    # thermal conductivity, W/mK
        self.spech  = spech   # volumetric specific heat, J/LK
    
    def R( self ):
        return self.thick / self.cond

In [21]:
def get_gypsum():
    return Material( name = 'GPB', thick = 0.0095, cond = 0.22, spech =  830.0 )

In [22]:
def get_plywood():
    return Material( name = 'PED', thick = 0.012,  cond = 0.16, spech =  720.0 )

In [23]:
def get_concrete():
    return Material( name = 'RC',  thick = 0.120,  cond = 1.60, spech = 2000.0 )

In [24]:
def convert_u_value_to_spec(structure, typeOfPart, U):
    # structure as string : Wood, Steel, RC, Other
    # typeOfPart as string: Ceiling, Wall, Floor, BoundaryCeiling, BoundaryWall, BoundaryFloor
    # U: heat transfer coefficient (W/m2K)
    s = structure
    t = typeOfPart
    R = 1 / U
    
    Ro, Ri = {
        'Roof'            : ( 0.040, 0.090 ),
        'Ceiling'         : ( 0.090, 0.090 ),
        'Wall'            : ( 0.040, 0.110 ),
        'Floor'           : ( 0.040, 0.150 ),
        'BoundaryCeiling' : ( 0.090, 0.090 ),
        'BoundaryWall'    : ( 0.110, 0.110 ),
        'BoundaryFloor'   : ( 0.150, 0.150 ),
    }[t]
    
    mGypsum   = get_gypsum()
    mPlywood  = get_plywood()
    mConcrete = get_concrete()
    
    ins_name, ins_cond, ins_spech = 'GW16K', 0.045, 13.0
    
    def d( m ):
        return ( math.floor( max(0,( R -( Ro + m.R() + Ri )) * ins_cond ) * 1000 ) ) / 1000
    
    if ( s in ['wood', 'steel', 'other'] ):
        m = {
            'Roof'            : mGypsum,
            'Ceiling'         : mGypsum,
            'Wall'            : mGypsum,
            'Floor'           : mPlywood,
            'BoundaryWall'    : mGypsum,
            'BoundaryFloor'   : mPlywood,
            'BoundaryCeiling' : mGypsum,
        }[t]
        Layer= [ Material( name = ins_name, thick = d(m), cond = ins_cond, spech = ins_spech ), m ]
    elif s == 'RC':
        m = mConcrete
        Layer= [ m, Material( name = ins_name, thick = d(m), cond = ins_cond, spech = ins_spech ) ]    
    else:
        raise ValueError
        
    return Layer

In [25]:
def make_layers_from_class(layers) :
    
    return [{
            'name'  : x.name,
            'thick' : x.thick,
            'cond'  : x.cond,
            'specH' : x.spech
    } for x in layers]

In [26]:
def get_isincontactwithoutsideair_for_simplifiedinput(type):
    
    if type == 'Roof' or type == 'Wall' :
        IsInContactWithOutsideAir = True
    elif type == 'Floor' :
        IsInContactWithOutsideAir = False
    else :
        raise ValueError
    
    return IsInContactWithOutsideAir

In [27]:
def make_d_walls_for_simplifiedinput(d_walls):
    
    for x in d_walls :
        
        x['IsInContactWithOutsideAir'] = get_isincontactwithoutsideair_for_simplifiedinput(x['type'])
        
        # U値入力では壁体構造が不明のため熱容量の小さい構造を仮定
        x['Layers'] = make_layers_from_class(convert_u_value_to_spec('wood', x['type'], x['UA']))
    
    return d_walls

In [28]:
def get_isincontactwithoutsideair_for_inputUA(direction):
    
    IsInContactWithOutsideAir = {
        'Top':     True,
        'N':       True,
        'NE':      True,
        'E':       True,
        'SE':      True,
        'S':       True,
        'SW':      True,
        'W':       True,
        'NW':      True,
        'Bottom':  True,
        'ClosedSpace':     False,
        'OpenBackFloor':   False,
        'ResidenceSpace':  False,
        'ClosedBackFloor': False        
    }[direction]
    
    return IsInContactWithOutsideAir

In [29]:
def make_d_wall_for_inputUA(d_wall):
    
    d_wall['IsInContactWithOutsideAir'] = get_isincontactwithoutsideair_for_inputUA(d_wall['directioin'])
    d_wall['Layers'] = convert_u_value_to_spec(d_wall['structure'], d_wall['type'], d_wall['UA'])
    
    return d_wall

### 詳細計算法の場合の壁体の設定

#### Outline

Partsの複数の要素を複数の部位に分離する。

#### Function

In [30]:
def make_layer(name, thick, cond, specH, **kwargs) :
    
    return {
            'name'  : name,
            'thick' : thick,
            'cond'  : cond,
            'specH' : specH
    }

In [31]:
def make_d_wall_for_inputalldetails(d_wall):

    d_walls = []
    for x in d_wall['Parts'] :
        d_walls.append({
            'name':      d_wall['name'],
            'direction': d_wall['direction'],
            'area':      d_wall['area']*x['AreaRatio'],
            'space':     d_wall['space'],
            'type':      d_wall['type'],
            'structure':     d_wall['structure'],        
            'IsSunshadeInput': d_wall['IsSunshadeInput'],
            'IsInContactWithOutsideAir': d_wall['IsInContactWithOutsideAir'],
            'Layers': [make_layer(**layer) for layer in x['Layers']]
        })
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
    
    return d_walls

### 簡略計算方法①：面積比率法（充填断熱する場合又は充填断熱し付加断熱する場合）の場合の壁体の設定 

#### Outline

Partsの複数の要素を複数の部位に分離する。  
面積比率は、木造住宅の建て方及び構法／工法の種類等に応じて与えられる。

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  

#### Outline

<div style="text-align: center;"> 表3-1 木造における床の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・床梁工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（大引間に断熱）  | 0.85 | 0.15 |
| 軸組構法・束立大引工法（根太間及び大引間に断熱）  |  |  |
| 軸組構法・剛床工法  | 0.85 | 0.15 |
| 軸組構法・床梁土台同面工法（根太間に断熱）  | 0.70 | 0.30 |
| 枠組壁工法（根太間に断熱）  | 0.87 | 0.13 |

<div style="text-align: center;"> 表3-2 軸組構法の束立大引工法において根太間及び大引間に断熱する場合の床の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（根太間断熱材＋大引間断熱材） | 断熱部分＋熱橋部分<br>（根太間断熱材＋大引材等） | 断熱部分＋熱橋部分<br>（根太材＋大引間断熱材） | 熱橋部分<br>（根太材＋大引材等） |
|---|---|---|---|---|
|  軸組構法の束立大引工法において<br>根太間及び大引間に断熱する場合 | 0.72 | 0.12 | 0.13 | 0.03 |

#### Function

In [32]:
def get_area_ratio_wood_floor(FloorConstructionMethod):
    # FloorConstructionMethod: 床の工法種類
    # 'FrameBeamInsuljoist'（軸組構法・床梁工法（根太間に断熱））/'FrameFootingInsuljoist'（軸組構法・束立大引工法（根太間に断熱））/
    # 'FrameFootingInsulsleeper'（軸組構法・束立大引工法（大引間に断熱））/
    # 'FrameFootingInsuljoistSleeper'（軸組構法・束立大引工法（根太間及び大引間に断熱））/
    # 'FrameRigidfloor'（軸組構法・剛床工法）/'FrameBeambaseInsuljoist'（軸組構法・床梁土台同面工法（根太間に断熱））/
    # 'WallInsuljoist'（枠組壁工法（根太間に断熱））
    
    if FloorConstructionMethod == 'FrameBeamInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsulsleeper' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameFootingInsuljoistSleeper' :
        area_ratio_wood_floor = {
            'InsulJoitandsleeper' : 0.72,
            'InsuljoistHeatbridgesleeper' : 0.12,
            'InsulsleeperHeatbridgejoist' : 0.13,
            'HeatbridgeJoistandsleeper' : 0.03
        }
    elif FloorConstructionMethod == 'FrameRigidfloor' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameBeambaseInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.70, 'Heatbridge' : 0.30}
    elif FloorConstructionMethod == 'WallInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.87, 'Heatbridge' : 0.13}
    else :
        raise ValueError
    
    return area_ratio_wood_floor

#### Outline

<div style="text-align: center;"> 表4-1 木造における外壁（界壁）の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・柱、間柱間に断熱 | 0.83 | 0.17 |
| 軸組構法・柱、間柱間に断熱し付加断熱 |  |  |
| 枠組壁工法・たて枠間に断熱 | 0.77 | 0.23 |
| 枠組壁工法・たて枠間に断熱＋付加断熱 |  |  |

<div style="text-align: center;"> 表4-2 軸組構法において柱・間柱間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（柱・間柱間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（柱・間柱間断熱材＋付加断熱層内熱橋部分） | 断熱部分＋熱橋部分<br>（構造部材等＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部分） |
|---|---|---|---|---|
| 付加断熱層内熱橋<br>部分が「横下地」 | 0.75 | 0.08 | 0.12 | 0.05 |
| 付加断熱層内熱橋<br>部分が「縦下地」 | 0.79 | 0.04 | 0.04 | 0.13 |

<div style="text-align: center;"> 表4-3 枠組壁工法においてたて枠間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（充填断熱材＋付加断熱材） | 断熱部分＋熱橋部<br>（充填断熱材＋付加断熱層内熱橋部） | 断熱部分＋熱橋部<br>（構造部材等＋付加断熱材） | 断熱部分＋熱橋部<br>（まぐさ＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部） | 熱橋部分<br>（まぐさ＋付加断熱層内熱橋部） |
|---|---|---|---|---|---|---|
| 付加断熱層内熱橋部分が「横下地」 | 0.69 | 0.08 | 0.14 | 0.02 | 0.06 | 0.01 |
| 付加断熱層内熱橋部分が「縦下地」 | 0.76 | 0.01 | 　 | 0.02 | 0.20 | 0.01 |

#### Function

In [33]:
def get_area_ratio_wood_wall(WallConstructionMethod):
    # WallConstructionMethod: 床の工法種類
    # 'FrameInsulcolumn'（軸組構法・柱、間柱間に断熱）/
    # 'FrameInsuladdBackhorizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'FrameInsuladdBackvertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））/
    # 'WallInsuljamb'（枠組壁工法・たて枠間に断熱）/
    # 'WallInsuladdBackhorizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'WallInsuladdBackvertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
    
    if WallConstructionMethod == 'FrameInsulcolumn' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    elif WallConstructionMethod == 'FrameInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'InsulColumnandadd' : 0.75,
            'InsulcolumnHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.12,
            'HeatbridgeMemberandadd' : 0.05
        }
    elif WallConstructionMethod == 'FrameInsuladdBackvertical' :
        area_ratio_wood_wall = {
            'InsulColumnandadd' : 0.79,
            'InsulcolumnHeatbridgeadd' : 0.04,
            'InsuladdHeatbridgemember' : 0.04,
            'HeatbridgeMemberandadd' : 0.13
        }
    elif WallConstructionMethod == 'WallInsuljamb' :
        area_ratio_wood_wall = {'Insulation' : 0.77, 'Heatbridge' : 0.23}
    elif WallConstructionMethod == 'WallInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'InsulFilledandadd' : 0.69,
            'InsulfilledHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.14,
            'InsuladdHeatbridgelintel' : 0.02,
            'HeatbridgeMemberandadd' : 0.06,
            'HeatbridgeLintelandadd' : 0.01
        }
    elif WallConstructionMethod == 'WallInsuladdBackvertical' :
        area_ratio_wood_wall = {        
            'InsulFilledandadd' : 0.76,
            'InsulfilledHeatbridgeadd' : 0.01,
            'InsuladdHeatbridgemember' : 0.00,
            'InsuladdHeatbridgelintel' : 0.02,
            'HeatbridgeMemberandadd' : 0.20,
            'HeatbridgeLintelandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_wall

#### Outline

<div style="text-align: center;"> 表5 木造における天井の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 桁・梁間に断熱する場合 | 0.87 | 0.13 |

#### Function

In [34]:
def get_area_ratio_wood_ceiling(CeilingConstructionMethod):
    # CeilingConstructionMethod: 天井の工法種類
    # 'Insulbeam'（桁・梁間に断熱）
    
    if CeilingConstructionMethod == 'Insulbeam' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    else :
        raise ValueError

    return area_ratio_wood_ceiling

#### Outline

<div style="text-align: center;"> 表6-1 木造における屋根の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| たるき間に断熱する場合  | 0.86 | 0.14 |

<div style="text-align: center;"> 表6-2 木造においてたるき間に断熱し付加断熱（横下地）する場合の屋根の面積比率  </div>

| 工法の種類等 | 断熱部分<br>（たる木間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（たる木間断熱材＋付加断熱層内熱橋部（下地たる木）） | 断熱部分＋熱橋部分<br>（構造部材＋付加断熱材） | 熱橋部分<br>（構造部材＋付加断熱層内熱橋部（下地たる木）） |
|---|---|---|---|---|
| たるき間に断熱し付加断熱（横下地）する場合  | 0.79 | 0.08 | 0.12 | 0.01 |

#### Function

In [35]:
def get_area_ratio_wood_roof(RoofConstructionMethod):
    # RoofConstructionMethod : 屋根の工法種類
    # 'Insulrafter'（たるき間に断熱）/'InsuladdBackhorizontal'（たるき間に断熱＋付加断熱（横下地）））
    
    if RoofConstructionMethod == 'Insulrafter' :
        area_ratio_wood_roof = {'Insulation' : 0.86, 'Heatbridge' : 0.14}
    elif RoofConstructionMethod == 'InsuladdBackhorizontal' :
        area_ratio_wood_roof = {
            'InsulRafterandadd' : 0.79,
            'InsulrafterHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.12,
            'HeatbridgeMemberandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_roof

#### Function

In [36]:
def get_area_ratio_wood(d_wall):
    
    if d_wall['Type'] == 'Roof' :
        return get_area_ratio_wood_roof(d_wall['RoofConstructionMethod'])
    elif d_wall['Type'] == 'Ceiling' or d_wall['Type'] == 'BoundaryCeiling' :
        return get_area_ratio_wood_ceiling(d_wall['CeilingConstructionMethod'])        
    elif d_wall['Type'] == 'Wall' or d_wall['Type'] == 'BoundaryWall' :
        return get_area_ratio_wood_wall(d_wall['WallConstructionMethod'])            
    elif d_wall['Type'] == 'Floor' :
        return get_area_ratio_wood_floor(d_wall['FloorConstructionMethod'])    
    else :
        raise ValueError        

In [37]:
def make_d_wall_for_inputalllayers(d_wall):
    
    area_ratio = get_area_ratio_wood(d_wall)

    d_walls = []
    for x in d_wall['Parts'] :
        d_walls.append({
            'name':      d_wall['name'],
            'direction': d_wall['direction'],
            'area':      d_wall['area']*area_ratio[x],
            'space':     d_wall['space'],
            'type':      d_wall['type'],
            'structure':     d_wall['structure'],        
            'IsSunshadeInput': d_wall['IsSunshadeInput'],
            'IsInContactWithOutsideAir': d_wall['IsInContactWithOutsideAir'],
            'Layers': [make_layer(**layer) for layer in x['Layers']]
        })
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
    
    return d_walls

### 簡略計算方法②：熱貫流率補正法、鉄骨造の場合の壁体の設定 

#### Outline

補正熱貫流率より部位の平均熱貫流率を算出する。  
部位の平均熱貫流率に一致するよう、一般部の層構成を調整する。

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  

#### Outline

<div style="text-align: center;"> 表7 木造における一般部位の断熱工法等に応じた補正熱貫流率 </div>

| 部位 | 断熱工法等 | 軸組構法補正熱貫流率 | 枠組壁構法補正熱貫流率 |
|---|---|---|---|
| 床 | 　 | 0.13 | 0.08 |
| 上階側界床 | 　 | 0.13 | 0.08 |
| 下階側界床 | 　 | 0.13 | 0.08 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）する場合 | 0.09 | 0.13 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）し付加断熱する場合 | 0.09 | 0.13 |

| 部位 | 断熱工法等 | 補正熱貫流率 |
|---|---|---|
| 外壁・界壁 | 土壁で外張断熱の場合 | 0.04 |
| 外壁・界壁 | 土壁以外で外張断熱の場合 | 0.02 |
| 天井 | 桁・梁間に断熱する場合 | 0.05 |
| 屋根 | 充填断熱（たるき間に断熱）する場合 | 0.11 |
| 屋根 | 充填断熱（たるき間に断熱）し付加断熱する場合 | 0.11 |
| 屋根 | 外張断熱工法 | 0.02 |

#### Function

In [38]:
def get_Ur_wood(d_wall):
    # Type : 部位種別
    # ConstructionMethod : 工法種類
    # 'FrameInsulcolumn'（軸組構法）/'WallInsul'（枠組壁構法）
    
    if d_wall['Type'] == 'Floor' or d_wall['Type'] == 'BoundaryCeiling' or d_wall['Type'] == 'BoundaryFloor' :
        if d_wall['ConstructionMethod'] == 'FrameInsulcolumn' :
            Ur = 0.13
        elif d_wall['ConstructionMethod'] == 'WallInsul' :
            Ur = 0.08
        else :
            raise ValueError

    # WallInsulationMethod : 壁の断熱種類
    # 'InsulColumn'（充填断熱（柱・間柱間に断熱））/'InsulColumnandadd'（充填断熱（柱・間柱間に断熱）+付加断熱）/
    # 'OuterInsulMudWall'（土壁で外張断熱）/'OuterInsul'（土壁以外で外張断熱）    
    elif d_wall['Type'] == 'Wall' or d_wall['Type'] == 'BoundaryWall' :
        if d_wall['WallInsulationMethod'] == 'InsulColumn' or d_wall['RoofInsulationMethod'] == 'InsulColumnandadd' :
            if d_wall['ConstructionMethod'] == 'FrameInsulcolumn' :
                Ur = 0.09
            elif d_wall['ConstructionMethod'] == 'WallInsul' :
                Ur = 0.13
            else :
                raise ValueError
        elif d_wall['WallInsulationMethod'] == 'OuterInsulMudWall' :
            Ur = 0.04
        elif d_wall['WallInsulationMethod'] == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    elif d_wall['Type'] == 'Ceiling' :
        Ur = 0.05

    # RoofInsulationMethod : 屋根の断熱種類
    # 'InsulRafter'（充填断熱（たるき間に断熱））/'InsulRafterandadd'（充填断熱（たるき間に断熱）+付加断熱）/'OuterInsul'（外張断熱工法）
    elif d_wall['Type'] == 'Roof' :
        if d_wall['RoofInsulationMethod'] == 'InsulRafter' or d_wall['RoofInsulationMethod'] == 'InsulRafterandadd' :
            Ur = 0.11
        elif d_wall['RoofInsulationMethod'] == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    else :
        raise ValueError
    
    return Ur

#### Outline

<div style="text-align: center;"> 表8 鉄骨造における一般部位の熱橋部分（柱及び梁以外）の仕様に応じた補正熱貫流率 </div>

| 「外装材＋断熱補強材」の熱抵抗(m<sup>2</sup>･K/W) | 補正熱貫流率ܷ |
|---|---|
| 1.7以上 | 0.00 |
| 1.7未満1.5以上 | 0.10 |
| 1.5未満1.3以上 | 0.13 |
| 1.3未満1.1以上 | 0.14 |
| 1.1未満0.9以上 | 0.18 |
| 0.9未満0.7以上 | 0.22 |
| 0.7未満0.5以上 | 0.40 |
| 0.5未満0.3以上 | 0.45 |
| 0.3未満0.1以上 | 0.60 |
| 0.1未満 | 0.70 |

#### Function

In [39]:
def get_Ur_steel(Rc):
    # Rer : 「外装材＋断熱補強材」の熱抵抗, m2K/W
    
    if Rc >= 1.7 :
        Ur = 0.00
    elif Rc >= 1.5 :
        Ur = 0.10
    elif Rc >= 1.3 :
        Ur = 0.13    
    elif Rc >= 1.1 :
        Ur = 0.14    
    elif Rc >= 0.9 :
        Ur = 0.18    
    elif Rc >= 0.7 :
        Ur = 0.22    
    elif Rc >= 0.5 :
        Ur = 0.40    
    elif Rc >= 0.3 :
        Ur = 0.45    
    elif Rc >= 0.1 :
        Ur = 0.60  
    elif Rc < 0.1 :
        Ur = 0.70
    else :
        raise ValueError

    return Ur

#### Outline

$$
\displaystyle
U_{i}= \frac{1}{R_{g,i}} + U_{r,i}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$R_{g,i}$: 一般部位$i$の断熱部分の熱抵抗, m<sup>2</sup>K/W  
$U_{r,i}$: 一般部位$i$の補正熱貫流率, W/(m<sup>2</sup>・K)  

#### Function

In [40]:
def calc_UA_from_Ur(d_wall):
    
    # 断熱部分の熱抵抗, m2K/W
    Ro = get_OutHeatTrans(d_wall['type'], d_wall['IsInContactWithOutsideAir']),
    Ri = get_InHetaTrans(d_wall['Type'])
    Rg = Ro + sum(x['Thick']/x['Cond'] for x in d_wall['Parts']['Insulation']['Layers']) + Ri
    
    # 補正熱貫流率
    if d_wall['Structure'] == 'Wood' :
        Ur = get_Ur_wood(d_wall)
    elif d_wall['Structure'] == 'Steel' :
        Ur = get_Ur_steel(d_wall['RCladding'])
    else :
        raise ValueError
    
    return 1/Rg + Ur

#### Outline

一般部位の熱貫流率$U_A$に一致するよう、一般部位の部分の層構成を調整する。  
－$U_A$に一致するまで、Layersの順番に厚を減らす  
－$U_A$に一致するまで、Layersの逆順に厚を減らす  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの順番）  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順）  

#### Function

In [41]:
# UAをもとに層構成を調整
# Layersの順番に厚を減らす
def get_layers_from_UA_acending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in layers :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                x['Thick'] = max(0, (1/UA - Ro - Ri - (sum(y['Thick']/y['Cond'] for y in layers)-x['Thick']/x['Cond']))*x['Cond'])
    
    return layers

In [42]:
# UAをもとに層構成を調整
# Layersの逆順に厚を減らす
def get_layers_from_UA_descending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in reversed(layers) :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                x['Thick'] = max(0, (1/UA - Ro - Ri - (sum(y['Thick']/y['Cond'] for y in layers)-x['Thick']/x['Cond']))*x['Cond'])
    
    return layers

In [43]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
def get_layers_from_UA_ins_acending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        l_copy = copy.deepcopy(l)
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(0,len(l_copy)) :
                if cond_temp == l_copy[y]['Cond'] :
                    sort_list.append(y)
                    l_copy[y]['Cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['Cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                R = Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers) - layers[x]['Thick']/layers[x]['Cond']
                layers[x]['Thick'] = max(0, (1/UA - R)*layers[x]['Cond'])
    
    return layers

In [44]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
def get_layers_from_UA_ins_descending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        l_copy = copy.deepcopy(l)
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(len(l_copy),0,-1) :
                if cond_temp == l_copy[y-1]['Cond'] :
                    sort_list.append(y-1)
                    l_copy[y-1]['Cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['Cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                R = Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers) - layers[x]['Thick']/layers[x]['Cond']
                layers[x]['Thick'] = max(0, (1/UA - R)*layers[x]['Cond'])
    
    return layers

In [45]:
def make_d_wall_for_inputUR(d_wall_init):
    
    UA = calc_UA_from_Ur(d_wall_init)
    
    d_wall = [{
        'name':      d_wall['name'],
        'direction': d_wall['direction'],
        'area':      d_wall['area'],
        'space':     d_wall['space'],
        'type':      d_wall['type'],
        'structure':     d_wall['structure'],        
        'IsSunshadeInput': d_wall['IsSunshadeInput'],
        'IsInContactWithOutsideAir': d_wall['IsInContactWithOutsideAir'],
        'Layers': get_layers_from_UA_ins_descending(d_wall_init['Parts'][0]['Layers'], UA, 
                                                    get_OutHeatTrans(d_wall['type'], d_wall['IsInContactWithOutsideAir']),
                                                    get_InHeatTrans_from_type(d_wall['type']))
    }]
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
            
    return d_wall

### RC造の場合の壁体の設定

#### Outline

情報を維持する。

#### Function

In [46]:
def make_d_wall_for_inputRC(d_wall_init):
    
    d_wall = [{
        'name':      d_wall['name'],
        'direction': d_wall['direction'],
        'area':      d_wall['area'],
        'space':     d_wall['space'],
        'type':      d_wall['type'],
        'structure':     d_wall['structure'],        
        'IsSunshadeInput': d_wall['IsSunshadeInput'],
        'IsInContactWithOutsideAir': d_wall['IsInContactWithOutsideAir'],
        'Layers':  [make_layer(**layer) for layer in d_wall['Parts'][0]['Layers']]
    }]
        
    if d_wall['IsSunshadeInput'] == True :
        for x in d_walls :
            x['Y1'] = d_wall['Y1']
            x['Y2'] = d_wall['Y2']
            x['Z'] = d_wall['Z']
            
    return d_wall

### Walls要素をRoomsに展開

#### Outline

Partsの複数の要素を複数の部位に分離する。

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [1]:
def integrate_walls_to_rooms(region, IsSimplifiedInput, d_walls_init, d_rooms):
    
    if IsSimplifiedInput == True :
        d_walls = make_d_walls_for_simplifiedinput(d_walls_init)
    else :
        d_walls = []
        for x in d_walls_init :
            # 直接入力
            if x['InputMethod'] == 'InputUA' :
                d_walls.append(make_d_wall_for_inputUA(x))
            # 木造 詳細計算法
            elif  d_wall['Structure'] == 'wood' and d_wall['InputMethodWood'] == 'InputAllDetails' :
                d_walls.extend(make_d_walls_for_inputalldetails(x))
            # 木造 簡略計算法①
            elif  d_wall['Structure'] == 'wood' and d_wall['InputMethodWood'] == 'InputAllLayers' :
                d_walls.extend(make_d_walls_for_inputalllayers(x))
            # 木造 簡略計算法②、鉄骨造
            elif (d_wall['Structure'] == 'wood' and d_wall['InputMethodWood'] == 'InputUR') or d_wall['Structure'] == 'Steel' :
                d_walls.extend(make_d_wall_for_inputUR(x))
            # RC造
            elif d_wall['Structure'] == 'RC' :
                d_walls.extend(make_d_wall_for_inputRC(x))       
            # その他
            elif d_wall['Structure'] == 'other' :
                raise ValueError        
            else :
                raise ValueError

    n = {'Roof': 1, 'Ceiling': 1, 'Wall': 1, 'Floor': 1, 'BoundaryCeiling': 1, 'BoundaryWall': 1, 'BoundaryFloor': 1}
    for x in d_walls :
        for y in d_rooms :
            if x['space'] == y['name'] :     
                y['Surface'].append({
                    'skin': True, 
                    'direction': get_direction_from_direction_for_outer_skin(x['direction'] if x['nextspace']=='Outdoor' else x['nextspace']),
                    'floor': True if x['direction'] == 'Bottom' else False,
                    'boundary': get_boundary_for_outer_skin(region, x['nextspace'], x['direction']),
                    'unsteady': True, 
                    'IsSoil': False,
                    'name': x['type'] + str(n[x['type']]), 
                    'area': x['area'], 
                    'Wall': {
                        'OutHeatTrans': get_OutHeatTrans(x['type'], x['IsInContactWithOutsideAir']),
                        'InHeatTrans': get_InHeatTrans_from_type(x['type']),
                        'Layers':x['Layers']
                    },
                    'sunbreak':{'D': x['Z'], 'hi': x['Y1'], 'WH': x['Y2']} if x['IsSunshadeInput'] == True else {}
                })
                n[x['type']] = n[x['type']] + 1
    
    return d_rooms

### InnerWalls要素をRoomsに展開

#### Outline

床下空間上の床、室上の床を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [1]:
def integrate_innerwalls_to_rooms(region, d_innerwalls, d_rooms):
    
    n = {'InnerCeiling': 1, 'InnerWall': 1, 'InnerFloor': 1, 'GroundFloor': 1}
    for x in d_innerwalls :
        for y in d_rooms :
            if x['space'] == y['name'] :    
                y['Surface'].append({
                    'skin': False, 
                    'direction': get_direction_from_direction_for_inner_skin(x['space'], x['type']),
                    'floor': True if (x['type'] == 'GroundFloor' or x['type'] == 'InnerFloor') else False,
                    'boundary': get_boundary_for_inner_skin(x['type'], x['nextspace']),
                    'unsteady': True,
                    'IsSoil': False,
                    'name': x['type'] + str(n[x['type']]), 
                    'area': x['area'], 
                    'Wall': {
                        'OutHeatTrans': get_OutHeatTrans(x['type'], False),
                        'InHeatTrans': get_InHeatTrans_from_type(x['type']),
                        'Layers':x['Layers']
                    },
                })
                n[x['type']] = n[x['type']] + 1
    
    return d_rooms

### Doors要素をRoomsに展開

#### Outline

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [49]:
def integrate_doors_to_rooms(region, IsSimplifiedInput, d_doors, d_rooms):
    
    n = 1
    for x in d_doors :
        for y in d_rooms :
            if x['space'] == y['name'] :
                y['Surface'].append({
                    'skin': True, 
                    'direction': get_direction_from_direction_for_outer_skin(x['direction']),
                    'floor': True if x['direction'] == 'Bottom' else False,
                    'boundary': get_boundary_for_outer_skin(region, x['nextspace'], x['direction']),
                    'unsteady': False, 
                    'name': 'Door' + str(n), 
                    'area': x['area'], 
                    'Windows': {
                        'Eta': 0.034*x['U'],
                        'Uw': x['U'],
                        'OutHeatTrans': 25,
                        'InHeatTrans': get_InHeatTrans_from_direction(x['direction'])
                    },
                    'sunbreak':{'D': x['Z'], 'hi': x['Y1'], 'WH': x['Y2']} if x['IsSunshadeInput'] == True else {}
                })
                n = n + 1
    
    return d_rooms

### Windows要素をRoomsに展開

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第四節　日射熱取得率  

#### Outline

<div style="text-align: center;"> 付録C　表1 ガラスの垂直面日射熱取得率 </div>

| ガラスの仕様 | 　 | 　 | 付属部材なし | 和障子 | 外付けブラインド |
|---|---|---|---|---|---|
| 三層複層 | 2枚以上のガラス表面にLow-E膜を使用したLow-E三層複層ガラス | 日射取得型 | 0.54 | 0.34 | 0.12 |
| 三層複層 | 2枚以上のガラス表面にLow-E膜を使用したLow-E三層複層ガラス | 日射遮蔽型 | 0.33 | 0.22 | 0.08 |
| 三層複層 | Low-E三層複層ガラス | 日射取得型 | 0.59 | 0.37 | 0.14 |
| 三層複層 | Low-E三層複層ガラス | 日射遮蔽型 | 0.37 | 0.25 | 0.10 |
| 三層複層 | 三層複層ガラス　| 　 | 0.72 | 0.38 | 0.18 |
| 二層複層 | Low-E二複層ガラス | 日射取得型 | 0.64 | 0.38 | 0.15 |
| 二層複層 | Low-E二複層ガラス | 日射遮蔽型 | 0.40 | 0.26 | 0.11 |
| 二層複層 | 二層複層ガラス | 　 | 0.79 | 0.38 | 0.17 |
| 二層複層 | 単板ガラス2枚を組み合わせたもの | 日射遮蔽型 | 0.79 | 0.38 | 0.17 |
| 単層 | 単板ガラス　| 　 | 0.88 | 0.38 | 0.19 |

In [50]:
def get_eta_from_specification(typeFrame, typeGlass, typeShade):
    
    f_frame = {'WoodOrResin': 0.72, 'Steel': 0.8}[typeFrame]
    
    eta = {
        '3WgG':            {'None': 0.54, 'Shoji': 0.34, 'ExtarnalBlind': 0.12}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス2枚）日射取得型
        '3WsG':            {'None': 0.33, 'Shoji': 0.22, 'ExtarnalBlind': 0.08}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス2枚）日射遮蔽型
        '3LgG':            {'None': 0.59, 'Shoji': 0.37, 'ExtarnalBlind': 0.14}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス1枚）日射取得型
        '3LsG':            {'None': 0.37, 'Shoji': 0.25, 'ExtarnalBlind': 0.10}, # 三層複層 Low-E三層複層ガラス（Low-Eガラス1枚）日射遮蔽型
        '2LgG':            {'None': 0.64, 'Shoji': 0.38, 'ExtarnalBlind': 0.15}, # 二層複層 Low-E複層ガラス日射取得型
        '2LsG':            {'None': 0.40, 'Shoji': 0.26, 'ExtarnalBlind': 0.11}, # 二層複層 Low-E複層ガラス日射遮蔽型
        '2FAheatreflect1': {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線反射ガラス1種
        '2FAheatreflect2': {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線反射ガラス2種
        '2FAheatreflect3': {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線反射ガラス3種
        '2FAheatabsorb2':  {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 遮熱複層ガラス熱線吸収板ガラス2種
        '2FAmulti':        {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 複層ガラス
        '2FAsingle':       {'None': 0.79, 'Shoji': 0.38, 'ExtarnalBlind': 0.17}, # 二層複層 単板ガラス2枚
        'Theatreflect1':   {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線反射ガラス1種
        'Theatreflect2':   {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線反射ガラス2種
        'Theatreflect3':   {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線反射ガラス3種
        'Theatabsorb2':    {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}, # 単層 単板ガラス熱線吸収板ガラス2種
        'Tsingle':         {'None': 0.88, 'Shoji': 0.38, 'ExtarnalBlind': 0.19}  # 単層 単板ガラスその他
    }[typeGlass][typeShade]
    
    return eta*f_frame

In [51]:
def get_eta(IsSimplifiedInput, d_window):
    
    if IsSimplifiedInput == True :
        eta = numpy.average([d_window['EtaCooling'], d_window['EtaHeating']])
    elif d_windows['IsEtaValueInput'] == True :
        if d_window['TypeWindow'] == 'Single' :
            eta = d_window['Eta']
        if d_window['TypeWindow'] == 'Double' :
            eta = d_window['EtaInside']*d_window['EtaOutside']*1.06/ \
                  (0.72 if d_window['TypeFrameInside'] == 'WoodOrResin' and d_window['TypeFrameOutside'] == 'WoodOrResin' else 0.8)
    elif d_windows['IsEtaValueInput'] == False :
        if d_window['TypeWindow'] == 'Single' :
            eta = get_eta_from_specification(d_window['TypeFrame'],d_window['TypeGlass'],d_window['TypeShade'])
        if d_window['TypeWindow'] == 'Double' :
            eta = get_eta_from_specification(d_window['TypeFrameInside'],d_window['TypeGlassInside'],d_window['TypeShadeInside'])* \
                  get_eta_from_specification(d_window['TypeFrameOutside'],d_window['TypeGlassOutside'],d_window['TypeShadeOutside'])* \
                  1.06/(0.72 if d_window['TypeFrameInside'] == 'WoodOrResin' and d_window['TypeFrameOutside'] == 'WoodOrResin' else 0.8)
    else :
        raise ValueError
    
    return eta

### Windows要素をRoomsに展開

#### Outline

方位が'Bottom'である外皮の部位を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [52]:
def integrate_windows_to_rooms(region, IsSimplifiedInput, d_windows, d_rooms):
    
    n = 1
    for x in d_windows :
        for y in d_rooms :
            if x['space'] == y['name'] :
                y['Surface'].append({
                    'skin': True, 
                    'direction': get_direction_from_direction_for_outer_skin(x['direction']),
                    'floor': True if x['direction'] == 'Bottom' else False,
                    'boundary': get_boundary_for_outer_skin(region, x['nextspace'], x['direction']),
                    'unsteady': False, 
                    'name': 'Window' + str(n), 
                    'area': x['area'], 
                    'Windows': {
                        'Eta': get_eta(IsSimplifiedInput, x),
                        'Uw': x['UW'],
                        'OutHeatTrans': 25,
                        'InHeatTrans': get_InHeatTrans_from_direction(x['direction'])
                    },
                    'sunbreak':{'D': x['Z'], 'hi': x['Y1'], 'WH': x['Y2']} if x['IsSunshadeInput'] == True else {}
                })
                n = n + 1
    
    return d_rooms

### Heabridges要素をRoomsに展開

#### Function

In [53]:
def integrate_heatbridges_to_rooms(region, d_heatbridges, d_rooms):
    
    n = 1
    for x in d_heatbridges :
        for y in d_rooms :
            if x['space'] == y['name'] :
                y['LinearHeatBridge'].append({
                    'name': 'Heatbridge' + str(n), 
                    'LinearHeatTrans': x['psi'], 
                    'LinearHeatTransLength': x['length'],
                    'boundary1': get_boundary_for_outer_skin(region, x['nextspace1'], x['direction1']),
                    'boundary2': get_boundary_for_outer_skin(region, x['nextspace2'], x['direction2'])
                })
                n = n + 1
    
    return d_room

### EarthfloorPerimeters要素をRoomsに展開

#### Function

In [1]:
def integrate_earthfloorperimeters_to_rooms(region, d_earthfloorperimeters, d_rooms):
    
    n = 1
    for x in d_earthfloorperimeters :
        for y in d_rooms :
            if x['space'] == y['name'] :
                y['EarthfloorPerimeter'].append({
                    'name': 'EarthfloorPerimeter' + str(n), 
                    'LinearHeatTrans': x['psi'], 
                    'LinearHeatTransLength': x['length'],
                    'boundary': get_boundary_for_earthfloorperimeter(region, 
                                                                     x['direction'] if x['nextspace']=='Outdoor' else x['nextspace']) 
                })
                n = n + 1
    
    return d_rooms

### Earthfloors要素をRoomsに展開

#### Outline

土間床を、床と判定する。  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [55]:
def integrate_earthfloors_to_rooms(region, d_earthfloors, d_rooms):
    
    n = 1
    for x in d_earthfloors :
        for y in d_rooms :
            if x['space'] == y['name'] :                
                y['Surface'].append({
                    'skin': True, 
                    'direction': 'Upward',
                    'floor': True,
                    'boundary': 'AnnualAverage',
                    'unsteady': True, 
                    'IsSoil': True,
                    'name': 'Earthfloor' + str(n), 
                    'area': x['area'], 
                    'Wall': {
                        'InHeatTrans': get_InHeatTrans_from_type('Earthfloor'),
                        'Layers':[{},{}]
                    },
                })
                n = n + 1
    
    return d_rooms

## 3. Convert

#### Function

In [56]:
def convert(d):
    
    d_calc_input = {}
    d_calc_input['Gdata'] = convert_gdata(d)
    d_calc_input['Rooms'] = get_rooms_initial(d)
    
    if ('Walls' in d) == True :
        d_calc_input['Rooms'] = integrate_walls_to_rooms(d['Common']['Region'], d['Common']['IsSimplifiedInput'], 
                                                         d['Walls'], d_calc_input['Rooms'])

    if ('Doors' in d) == True :
        d_calc_input['Rooms'] = integrate_doors_to_rooms(d['Common']['Region'], d['Common']['IsSimplifiedInput'], 
                                                         d['Doors'], d_calc_input['Rooms'])

    if ('Windows' in d) == True :
        d_calc_input['Rooms'] = integrate_windows_to_rooms(d['Common']['Region'], d['Common']['IsSimplifiedInput'], 
                                                           d['Windows'], d_calc_input['Rooms'])

    if ('Heatbridges' in d) == True :
        d_calc_input['Rooms'] = integrate_heatbridges_to_rooms(d['Common']['Region'], d['Heatbridges'], d_calc_input['Rooms'])

    if ('EarthfloorPerimeters' in d) == True :
        d_calc_input['Rooms'] = integrate_earthfloorperimeters_to_rooms(d['Common']['Region'], d['EarthfloorPerimeters'], d_calc_input['Rooms'])

    if ('Earthfloors' in d) == True :
        d_calc_input['Rooms'] = integrate_earthfloors_to_rooms(d['Common']['Region'], d['Earthfloors'], d_calc_input['Rooms'])

    if ('InnerWalls' in d) == True :
        d_calc_input['Rooms'] = integrate_innerwalls_to_rooms(d['Common']['Region'], d['InnerWalls'], d_calc_input['Rooms'])

    return d_calc_input

#### Example

In [57]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': True,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 30.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        {'name': 'Roof_main', 'direction': 'Top', 'area': 16.95, 'space': 'main', 'type': 'Roof',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Roof_other', 'direction': 'Top', 'area': 16.95, 'space': 'other', 'type': 'Roof',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Roof_nonliving', 'direction': 'Top', 'area': 33.9, 'space': 'nonliving', 'type': 'Roof',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Wall_SW_main', 'direction': 'SW', 'area': 10.1575, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_other', 'direction': 'SW', 'area': 10.1575, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_nonliving', 'direction': 'SW', 'area': 20.315, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_main', 'direction': 'NW', 'area': 7.4575, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_other', 'direction': 'NW', 'area': 7.4575, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_nonliving', 'direction': 'NW', 'area': 14.915, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_main', 'direction': 'NE', 'area': 15.9725, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_other', 'direction': 'NE', 'area': 15.9725, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_nonliving', 'direction': 'NE', 'area': 31.945, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_main', 'direction': 'SE', 'area': 7.4275, 'space': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_other', 'direction': 'SE', 'area': 7.4275, 'space': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_nonliving', 'direction': 'SE', 'area': 14.855, 'space': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Floor_main', 'direction': 'Bottom', 'area': 15.0175, 'space': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_other', 'direction': 'Bottom', 'area': 15.0175, 'space': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_nonliving', 'direction': 'Bottom', 'area': 30.035, 'space': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_main', 'direction': 'Bottom', 'area': 1.1025, 'space': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_other', 'direction': 'Bottom', 'area': 1.1025, 'space': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_nonliving', 'direction': 'Bottom', 'area': 2.205, 'space': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27}
    ],
    'Windows': [
        {'name': 'WindowSW_main', 'direction': 'SW', 'area': 7.5625, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSW_other', 'direction': 'SW', 'area': 7.5625, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSW_nonliving', 'direction': 'SW', 'area': 15.125, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowNW_main', 'direction': 'NW', 'area': 0.7925, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNW_other', 'direction': 'NW', 'area': 0.7925, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNW_nonliving', 'direction': 'NW', 'area': 1.585, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNE_main', 'direction': 'NE', 'area': 1.21, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNE_other', 'direction': 'NE', 'area': 1.21, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowNE_nonliving', 'direction': 'NE', 'area': 2.42, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.316},
        {'name': 'WindowSE_main', 'direction': 'SE', 'area': 1.4575, 'space': 'main', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSE_other', 'direction': 'SE', 'area': 1.4575, 'space': 'other', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.348},
        {'name': 'WindowSE_nonliving', 'direction': 'SE', 'area': 2.915, 'space': 'nonliving', 'UW': 3.49,
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': 0.348}
    ],
    'Doors': [
        {'name': 'DoorNW_main', 'direction': 'NW', 'area': 0.63, 'space': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_other', 'direction': 'NW', 'area': 0.63, 'space': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_nonliving', 'direction': 'NW', 'area': 1.26, 'space': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_main', 'direction': 'NE', 'area': 0.54, 'space': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_other', 'direction': 'NE', 'area': 0.54, 'space': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_nonliving', 'direction': 'NE', 'area': 1.08, 'space': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False}
    ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'Entrance_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 1.83, 'name': 'Entrance_NE', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'OpenBackFloor', 'length': 4.25, 'name': 'Entrance_floor', 'psi': 1.8, 'space': 'underfloor'}
    ],
    'Earthfloors': [
        {'name': 'Earthfloor', 'area': 3.24, 'space': 'underfloor'}
    ],
    'InnerWalls': [
        {'name': 'GroundFloor_main', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main', 'nextspace': 'underfloor'},
        {'name': 'GroundFloor_main', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'underfloor', 'nextspace': 'main'},
        {'name': 'GroundFloor_other', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other', 'nextspace': 'underfloor'},
        {'name': 'GroundFloor_other', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 0.81,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'underfloor', 'nextspace': 'other'},
        {'name': 'GroundFloor_nonliving', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 1.62,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'nonliving', 'nextspace': 'underfloor'},
        {'name': 'GroundFloor_nonliving', 'type': 'GroundFloor', 'direction': 'Horizontal', 'area': 1.62,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'underfloor', 'nextspace': 'nonliving'},
        {'name': 'InnerFloor_main', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main', 'nextspace': 'other'},
        {'name': 'InnerFloor_main', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main', 'nextspace': 'nonliving'},
        {'name': 'InnerFloor_other', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other', 'nextspace': 'main'},
        {'name': 'InnerFloor_other', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other', 'nextspace': 'nonliving'},
        {'name': 'InnerFloor_nonliving', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'nonliving', 'nextspace': 'main'},
        {'name': 'InnerFloor_nonliving', 'type': 'InnerFloor', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'nonliving', 'nextspace': 'other'},
        {'name': 'InnerCeiling_main', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main', 'nextspace': 'other'},
        {'name': 'InnerCeiling_main', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'main', 'nextspace': 'nonliving'},
        {'name': 'InnerCeiling_other', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 10.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 
                     720.0}],
         'space': 'other', 'nextspace': 'main'},
        {'name': 'InnerCeiling_other', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 20.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'other', 'nextspace': 'nonliving'},
        {'name': 'InnerCeiling_nonliving', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'nonliving', 'nextspace': 'main'},
        {'name': 'InnerCeiling_nonliving', 'type': 'InnerCeiling', 'direction': 'Horizontal', 'area': 30.0,
         'Layers': [{'name': 'PED', 'cond': 0.16, 'thick': 0.012, 'specH': 720.0}],
         'space': 'nonliving', 'nextspace': 'other'}
    ]
}

with open('C:/Users/PC/Documents/e7_inputdataconvert/Input.json', 'w') as f:
    json.dump(convert(d), f, indent=4)